# Notebook for data visualization of the processed HFO data from human SEEGs

## Add the parent directory to the path to detect the utils module

In [1]:
import os
import sys

# Add the parent directory to the path so it detects the utils module
module_path = os.path.abspath(os.path.join('..'))   
if module_path not in sys.path:
    sys.path.append(module_path)

## Load the data from the .npy file

In [2]:
import numpy as np

file_name = "seeg_synthetic_humans.npy"
recorded_data = np.load(file_name)

print("Data shape: ", recorded_data.shape)
print("First time steps: ", recorded_data[:10])

Data shape:  (245760, 960)
First time steps:  [[ 3.2352024e-01 -1.3235390e+00 -5.9668809e-01 ... -1.9608999e+00
  -1.9769822e-01 -1.2078454e+00]
 [-6.9759099e-04 -3.5122361e+00 -4.8766956e-01 ... -5.8757830e+00
  -7.4400985e-01 -5.1096064e-01]
 [ 1.9026639e+00 -5.6726017e+00  9.8274893e-01 ... -6.6182971e+00
  -8.3053267e-01 -8.1596655e-01]
 ...
 [ 3.2172418e+00 -8.4650068e+00  1.5216088e+00 ... -4.1081657e+00
   2.0085973e-01 -4.7539668e+00]
 [ 1.7725919e+00 -9.4744024e+00  1.6776791e+00 ... -4.1469693e+00
   1.6412770e+00 -3.4672713e+00]
 [ 7.8109097e-01 -1.0500931e+01  2.3717029e+00 ... -5.1762242e+00
   1.0715837e+00 -4.4489903e+00]]


## Define some important parameters of the input data

In [3]:
sampling_rate = 2048    # 2048 Hz
input_duration = 120 * (10**3)    # 120000 ms or 120 seconds
num_samples = recorded_data.shape[0]    # 2048 * 120 = 245760
num_channels = recorded_data.shape[1]   # 960

x_step = 1/sampling_rate * (10**3)  # 0.48828125 ms

## Plot the data in an interactive Line Plot

### Configurable parameters

In [4]:
# Channels to plot
min_channel = 0     #   Index of the first channel to plot
max_channel = 0     #   Index of the last channel to plot

In [6]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or x_step?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

# Create the y arrays for the voltage plot representing the voltage of each electrode
v_yarrays = []
# Add each channel
for i in range(min_channel, max_channel+1):  # TODO: Only adding 1 channel for now
    v_yarrays.append([val[i] for val in recorded_data])

# Create the plot
# List of tuples containing the y values and the legend label
hfo_y_arrays = [(voltage_val, f"Ch. {idx}") for idx, voltage_val in enumerate(v_yarrays)]

# Create the SEEG Voltage plot
hfo_plot = create_fig(
    title="SEEG Voltage dynamics of Synthetic data from Human recordings", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (μV)',
    x=x, 
    y_arrays=hfo_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    # y_range=Range1d(-0.05, 1.05)
)

# If there are more than 30 channels, hide the legend
if max_channel - min_channel + 1 > 30:
    # Hide the legend
    hfo_plot.legend.visible = False


## Show the Plots assembled in a grid